In [3]:
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
)

ImportError: cannot import name 'NomicEmbedding' from 'llama_index.embeddings' (unknown location)

In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
nomic_api_key = "&lt;NOMIC_API_KEY&gt;"
embed_model = NomicEmbedding(
    api_key=nomic_api_key,
    model_name="nomic-embed-text-v1",
    task_type="search_document"
)
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, chunk_size=1024,
)
index = VectorStoreIndex.from_documents(
    documents=documents, service_context=service_context, show_progress=True
)
